# AmBe analysis leftover code

In [ ]:
# Spacial flow to truth selection

def flow2truth(h5flow, h5truth):
    # Match the flow hit to truth-level particle trajectory
    # Assuming that the trajectory of particles are lines
    # The output will be a list of file trajectory ids, each representing a truth level trajectory the flow level hit is spacially close to
    truth_match = []
    truth_match_evt = []
    truth_match_vert = []
    truth_match_pri = []
    tolerance = 0.001  # Spacial search tolerance
    for i in h5flow:
        #flow_coord.append((h5truth['x'], h5truth['y'], h5truth['z']))
        p_flow = np.array([i['x'], i['y'], i['z']])
        for j in h5truth:
            if np.linalg.norm(np.dot(j['xyz_start']-j['xyz_end'], p_flow-j['xyz_end']))/(np.linalg.norm(j['xyz_start']-j['xyz_end'])*np.linalg.norm(p_flow-j['xyz_end'])) >= 1.0-tolerance:
                truth_match.append(j['file_traj_id'])
                truth_match_evt.append(j['event_id'])
                truth_match_vert.append(j['vertex_id'])
                truth_match_pri.append(j['primary'])
                break
    print(f"Total number of trajectories in file: {max(h5truth['event_id'])}")
    print(f"Total number of events in file: {len(np.where(h5truth['primary']==True)[0])}")
    print(f"Length of file: {len(h5truth)}")
    return truth_match, truth_match_evt, truth_match_vert, truth_match_pri

In [ ]:
# Convert the format of the flow-truth match up data

flow2truth_evt = [int(i) for i in flow2truth_evt[0]]
flow2truth_vert = [int(i) for i in flow2truth_vert[0]]
flow2truth_pri = [int(i) for i in flow2truth_pri[0]]

In [ ]:
# Plot a histogram of the event id

plt.figure(figsize=(8, 6))
plt.hist(flow2truth_evt, color='purple', bins=50)
#plt.yscale('log')
plt.xlabel('Event ID')
plt.ylabel('Hits Count')
plt.title(r'Truth Events that Produced Hits')
plt.grid(True)
#plt.xlim((0,50))
plt.show() 

In [ ]:
# Produce a histogram of hits produced per event

# Generate the count first
hits_per_evt = [flow2truth_evt.count(i) for i in list(set(flow2truth_evt))]

In [ ]:
# Histogram of hits per event

plt.figure(figsize=(8, 6))
# Plot the hits per event, including the events with zero hits
plt.hist(np.append(np.array(hits_per_evt), np.zeros(90-len(hits_per_evt))), color='purple', bins=40)
#plt.yscale('log')
plt.xlabel('Hits')
plt.ylabel('Event Count')
plt.title(r'Hits per Event')
plt.grid(True)
#plt.ylim((0,20))
plt.show() 

## Code for old (spatial) backtracking scheme

In [ ]:
# Load AmBe source files
# Load all hdf5 files in the directory, output data categories of interest
# TODO: some files don't have the light key, figure out why

#truth_N_index = []
truth_N_multi = []
flow_hits = []
truth_N_energies = []
flow2truth_match = []
flow2truth_evt = []
flow2truth_vert = []
flow2truth_pri = []

hits_bt = []

for file in glob.iglob('../AmBe_Prod_V2/FLOW/*.hdf5'):
    truth_N_index = []
    with h5py.File(file, 'r') as h5file:
        temp_truth_index = get_index_truth(h5file['mc_truth/trajectories/data'], 2112, True)
        truth_N_index.extend(temp_truth_index)
        # Try out the new backtracking scheme
        # Find all events that have corresponding hits
        hits_bt.extend(alltruth2charge(h5file, temp_truth_index, 'calib_prompt_hits'))
        #truth_N_multi.extend(get_mutiplicity_truth(h5file['mc_truth/trajectories/data'], truth_N_index))
        #truth_N_energies.append(get_energy_truth(h5file['mc_truth/trajectories/data'], truth_N_index))
        #print(len(h5file['mc_truth/trajectories/data']))
        #temp_match, temp_match_evt, temp_match_vert, temp_match_pri = flow2truth(h5file['charge/calib_prompt_hits/data'], h5file['mc_truth/trajectories/data'])
        #flow2truth_match.append(temp_match)
        #flow2truth_evt.append(temp_match_evt)
        #flow2truth_vert.append(temp_match_vert)
        #flow2truth_pri.append(temp_match_pri)
        #print(len(flow2truth_match[0]))
        #print(len(h5file['charge/calib_prompt_hits/data']['x']))
        #break

In [ ]:
# Histogram of hits per event with new hits finding scheme

hitpevt_temp = np.append(np.array([len(i) for i in hits_bt]), np.zeros(90-len(hits_bt)))

plt.figure(figsize=(8, 6))
# Plot the hits per event, including the events with zero hits
plt.hist(hitpevt_temp, color='purple', bins=8)
#plt.yscale('log')
plt.xlabel('Hits')
plt.ylabel('Event Count')
plt.title(r'Hits per Event (From Primary Neutron)')
plt.grid(True)
#plt.ylim((0,20))
plt.show()

In [ ]:
# Energy deposited per event (including the events with zero hits)

Epevt_temp = np.append(np.array([sum(i['E']) for i in hits_bt]), np.zeros(90-len(hits_bt)))

plt.figure(figsize=(8, 6))
plt.hist(Epevt_temp, color='purple', bins=np.linspace(min(Epevt_temp), max(Epevt_temp), 10))
#plt.yscale('log')
plt.xlabel('MeV')
plt.ylabel('Event Count')
plt.title(r'Energy Deposited in Detector per Event (From Primary Neutron)')
plt.grid(True)
#plt.ylim((0,20))
plt.show()

In [ ]:
# Plot energy of all hits

Ehits_temp = []
for i in hits_bt:
    Ehits_temp.extend(i['E'])

plt.figure(figsize=(8, 6))
plt.hist(Ehits_temp, color='purple', bins=np.linspace(min(Ehits_temp), max(Ehits_temp), 20))
#plt.yscale('log')
plt.xlabel('MeV')
plt.ylabel('Hits Count')
plt.title(r'Energy per Hit (From Primary Neutron)')
plt.grid(True)
plt.xlim((0,0.9))
plt.show()

In [ ]:
Epevt_temp = np.append(np.array([sum(i['E']) for i in hits_bt]), np.zeros(90-len(hits_bt)))

plt.figure(figsize=(8, 6))
plt.hist(Epevt_temp, color='purple', bins=np.linspace(min(Epevt_temp), max(Epevt_temp), 10))
#plt.yscale('log')
plt.xlabel('MeV')
plt.ylabel('Event Count')
plt.title(r'Energy Deposited in Detector per Event')
plt.grid(True)
#plt.ylim((0,20))
plt.show()